In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd


# 从附件1中读取数据。
df_shipment = pd.read_excel("./附件表/附件1-商家历史出货量表.xlsx")



df_shipment.head()


,seller_no,product_no,warehouse_no,date,qty
0,seller_19,product_448,wh_30,2023-05-09,10
1,seller_19,product_448,wh_30,2023-04-17,14
2,seller_19,product_448,wh_30,2023-01-09,2
3,seller_19,product_448,wh_30,2023-01-20,1
4,seller_19,product_448,wh_30,2023-02-13,22


In [3]:
df_shipment['合并主键']=df_shipment['seller_no']+df_shipment['product_no']+df_shipment['warehouse_no']

In [4]:
import numpy as np

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    """
    计算1-WMAPE（Weighted Mean Absolute Percentage Error）。

    参数：
    y_true: 真实需求量的numpy数组或列表
    y_pred: 预测需求量的numpy数组或列表

    返回值：
    1-WMAPE
    """
    # 将输入转换为numpy数组
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # 计算APE
    ape = np.abs(y_true - y_pred)
    
    # 计算1-WMAPE
    wmape = 1 - np.sum(ape) / np.sum(y_true)
    
    return wmape

def getdata(x):
    try:
        return round(y_test[0][0])
    except:
        try:
            return round(y_test[0])
        except:
            return round(y_test[0][0][0])

In [5]:
df_shipment

,seller_no,product_no,warehouse_no,date,qty,合并主键
0,seller_19,product_448,wh_30,2023-05-09,10,seller_19product_448wh_30
1,seller_19,product_448,wh_30,2023-04-17,14,seller_19product_448wh_30
2,seller_19,product_448,wh_30,2023-01-09,2,seller_19product_448wh_30
3,seller_19,product_448,wh_30,2023-01-20,1,seller_19product_448wh_30
4,seller_19,product_448,wh_30,2023-02-13,22,seller_19product_448wh_30
...,...,...,...,...,...,...
331331,seller_31,product_1015,wh_52,2023-01-20,1,seller_31product_1015wh_52
331332,seller_31,product_1015,wh_52,2023-01-13,6,seller_31product_1015wh_52
331333,seller_31,product_1015,wh_52,2023-01-12,3,seller_31product_1015wh_52
331334,seller_31,product_1015,wh_52,2023-01-22,0,seller_31product_1015wh_52


In [6]:
dd=df_shipment['合并主键'].unique()

In [19]:
%%time
data = []

total_iterations = len(dd)
output_frequency = total_iterations // 100  # 每隔1%输出一次

for index, d in enumerate(dd):
    # 计算已完成的百分比
    progress_percentage = (index + 1) / total_iterations * 100

    # 如果达到输出百分比的条件，就输出一次
    if index % output_frequency == 0:
        print(f"Progress: {progress_percentage:.1f}%")

        
    try:
        
        temp=df_shipment[df_shipment['合并主键']==d]
        seller_no=temp['seller_no'].unique()[0]
        product_no=temp['product_no'].unique()[0]
        warehouse_no=temp['warehouse_no'].unique()[0]

        temp=temp[['date','qty']]
        # 根据日期排序
        temp['date'] = pd.to_datetime(temp['date'])
        temp = temp.sort_values(by='date')
        # 将数据按15天为一个周期进行合并，首先需要计算最近的日期与2023-05-15的差值，然后从这个差值开始合并
        days_diff = (temp["date"].max() - temp.iloc[-1,0]).days

        # 使用rolling方法合并数据，窗口大小为15
        temp['X'] = temp['qty'].rolling(window=15, min_periods=1).sum()

        # 根据差值筛选出合并后的数据
        df_grouped = temp.iloc[days_diff::15].reset_index(drop=True)


        temp=df_grouped[['X']].iloc[1:]
        temp['y']=temp['X'].shift(-1)
        temp=temp.dropna()

        X_train, X_test, y_train, y_test = train_test_split(temp['X'].values.reshape(-1, 1), temp['y'].values.reshape(-1, 1), test_size=0.1,shuffle=False )
        # 创建并训练各种模型
        models = {
            "Linear Regression": LinearRegression(),
            "Random Forest": RandomForestRegressor(),
            "Decision Tree": DecisionTreeRegressor(),
            "GBDT": GradientBoostingRegressor(),
            "MLP": MLPRegressor(),
            "XGBoost": xgb.XGBRegressor(objective='reg:squarederror')
        }

        best_model = None
        best_error = -1  # 最大值初始化为负数

        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            error  = weighted_mean_absolute_percentage_error(y_test, y_pred)


            if error > best_error:
                best_error = error
                best_model = model



        # 预测未来一个时间周期的销售量         	seller_no	product_no	warehouse_no
        try:
            future_sales = round(best_model.predict(y_test)[0]) 
        except:
            future_sales = round(best_model.predict(y_test)[0][0]) 
        data.append([seller_no,product_no,warehouse_no,future_sales,best_model,best_error])
    except:
        data.append([seller_no,product_no,warehouse_no,getdata(y_test),0,0])

Progress: 0.1%
Progress: 1.0%
Progress: 2.0%
Progress: 2.9%
Progress: 3.9%
Progress: 4.8%
Progress: 5.8%
Progress: 6.7%
Progress: 7.7%
Progress: 8.6%
Progress: 9.6%
Progress: 10.5%
Progress: 11.5%
Progress: 12.4%
Progress: 13.4%
Progress: 14.3%
Progress: 15.3%
Progress: 16.2%
Progress: 17.2%
Progress: 18.1%
Progress: 19.1%
Progress: 20.0%
Progress: 21.0%
Progress: 21.9%
Progress: 22.9%
Progress: 23.8%
Progress: 24.8%
Progress: 25.8%
Progress: 26.7%
Progress: 27.7%
Progress: 28.6%
Progress: 29.6%
Progress: 30.5%
Progress: 31.5%
Progress: 32.4%
Progress: 33.4%
Progress: 34.3%
Progress: 35.3%
Progress: 36.2%
Progress: 37.2%
Progress: 38.1%
Progress: 39.1%
Progress: 40.0%
Progress: 41.0%
Progress: 41.9%
Progress: 42.9%
Progress: 43.8%
Progress: 44.8%
Progress: 45.7%
Progress: 46.7%
Progress: 47.6%
Progress: 48.6%
Progress: 49.5%
Progress: 50.5%
Progress: 51.5%
Progress: 52.4%
Progress: 53.4%
Progress: 54.3%
Progress: 55.3%
Progress: 56.2%
Progress: 57.2%
Progress: 58.1%
Progress: 59.1%
Pro

In [20]:
dfoutput=pd.DataFrame(data)
dfoutput.columns=['seller_no','product_no','warehouse_no','future_sales','best_model','best_error']
dfoutput

,seller_no,product_no,warehouse_no,future_sales,best_model,best_error
0,seller_19,product_448,wh_30,161,MLPRegressor(),0.499425
1,seller_11,product_148,wh_1,279,MLPRegressor(),0.950086
2,seller_11,product_132,wh_16,23,MLPRegressor(),0.975640
3,seller_11,product_170,wh_9,71,"(DecisionTreeRegressor(max_features='auto', ra...",0.268571
4,seller_11,product_184,wh_18,182,MLPRegressor(),0.567084
...,...,...,...,...,...,...
1991,seller_32,product_1061,wh_1,118,"(DecisionTreeRegressor(max_features='auto', ra...",0.942203
1992,seller_32,product_1112,wh_29,286,"(DecisionTreeRegressor(max_features='auto', ra...",0.798861
1993,seller_15,product_830,wh_1,494,MLPRegressor(),0.879285
1994,seller_21,product_494,wh_33,9,MLPRegressor(),0.180941


In [21]:
dfoutput[dfoutput['best_model']==0]

,seller_no,product_no,warehouse_no,future_sales,best_model,best_error
88,seller_11,product_1798,wh_16,7,0,0.0
118,seller_3,product_1638,wh_1,0,0,0.0
149,seller_3,product_1638,wh_31,0,0,0.0
294,seller_11,product_136,wh_18,3,0,0.0
426,seller_5,product_1541,wh_7,0,0,0.0
483,seller_24,product_942,wh_49,0,0,0.0
622,seller_27,product_648,wh_1,20,0,0.0
648,seller_11,product_136,wh_9,5,0,0.0
654,seller_27,product_676,wh_25,3,0,0.0
669,seller_33,product_1351,wh_29,580,0,0.0


In [22]:
dfoutput[dfoutput['best_model']!=0]['best_error'].mean()

0.8274578693809325

In [23]:
data1=pd.read_excel('./结果表/结果表1-预测结果表.xlsx')
data1

,seller_no,product_no,warehouse_no,date,forecast_qty
0,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,162
1,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,279
2,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,23
3,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,72
4,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,183
...,...,...,...,...,...
1991,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,134
1992,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,285
1993,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,490
1994,seller_19,product_448,wh_30,2023-05-16 至 2023-05-30,9


In [24]:
data1['seller_no']=dfoutput['seller_no']
data1['product_no']=dfoutput['product_no']
data1['warehouse_no']=dfoutput['warehouse_no']
data1['date']='2023-05-16 至 2023-05-30'
data1['forecast_qty']=dfoutput['future_sales']

In [25]:
data1.to_excel('./结果表/结果表1-预测结果表.xlsx',index=None)